In [2]:
# import numpy as np
# import pandas as pd
# import json

# ## SET PATH to code directory in the project folder
# code_path = 'C:\\evdoxiataka\\effi_user_study1\\code'
# import sys  
# sys.path.insert(1, code_path)

# ## SET folder to project directory
# folder="C:\\evdoxiataka\\effi_user_study1\\"

# from utils.utils import *
# from utils.training import oneoff_training_evaluation, iml_training_evaluation
# from utils.visualization import *

# import matplotlib
# import matplotlib.pyplot as plt

# import warnings
# from pandas.errors import SettingWithCopyWarning
# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# ## Any of the training set's attributes could be added here, protected or not, sensitive or not.
# ## We evaluated fairness for the following attributes
# sensitive_attrs = ["CODE_GENDER",
#                    "NAME_FAMILY_STATUS",
#                    "AGE",
#                    "REGION_RATING_CLIENT",
#                    "REGION_RATING_CLIENT_W_CITY",
#                    "OBS_30_CNT_SOCIAL_CIRCLE",
#                    "DEF_30_CNT_SOCIAL_CIRCLE",
#                    "OBS_60_CNT_SOCIAL_CIRCLE",
#                    "DEF_60_CNT_SOCIAL_CIRCLE"]
# fs = 0.5

# colors = ['r','g','b','orange','purple', 'y','cyan','magenta','tab:blue']

import numpy as np
import pandas as pd
import ast

########################SET folder to path of code directory in the project folder#####################
## SET PATH to code directory in the project folder
code_path = "C:\\evdoxiataka\\effi_user_study1\\code"
import sys  
sys.path.insert(1, code_path)

########################SET folder to project directory path#####################
folder= "C:\\evdoxiataka\\effi_user_study1\\"

from utils.data_loading import get_all_participants_logs, get_all_participants_feedback
from utils.utils import attributes_names_mapping, manipulate_categ_values, binning
from utils.training import oneoff_training_evaluation, iml_training_evaluation

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

sensitive_attrs = list(attributes_names_mapping.keys())
fs = 0.5

In [23]:
## load training, validation and test sets
train_df_train = pd.read_csv(folder+'data/processed_data/train_df_train.csv', delimiter=',')
train_df_test = pd.read_csv(folder+'data/processed_data/train_df_test.csv', delimiter=',')
test_df = pd.read_csv(folder+'data/processed_data/test_df_final.csv', delimiter=',')
X_train_original = train_df_train.loc[:, ~train_df_train.columns.isin(['TARGET', 'SK_ID_CURR'])]
y_train_original = train_df_train.loc[:, train_df_train.columns == 'TARGET']
X_test_original = train_df_test.loc[:, ~train_df_test.columns.isin(['TARGET', 'SK_ID_CURR'])] ## dataset used for accuracy
y_test_original = train_df_test.loc[:, train_df_test.columns == 'TARGET']

## attributes used in training in correct order
training_attr_in_order = X_train_original.columns.tolist()
training_attr_descriptive_in_order = [attributes_names_mapping[attr[:-3]] if attr.endswith('_LE') else attributes_names_mapping[attr] for attr in training_attr_in_order]

## load original test set to be used for fairness metrics
train_df = pd.read_csv(folder+'data/processed_data/train_df.csv', delimiter=',')
test_df_app_ids = train_df_test['SK_ID_CURR'].tolist()
train_df_test_or = train_df[train_df['SK_ID_CURR'].isin(test_df_app_ids)]
train_df_test_or = train_df_test_or.reset_index().set_index('SK_ID_CURR').loc[test_df_app_ids].reset_index().set_index('index').rename_axis(None)
manipulate_categ_values(train_df_test_or)
train_df_test_bin = binning(train_df_test_or, train_df_test)

In [43]:
## load application data shown to participants through the UI
applications_df = pd.read_csv(folder+"data/processed_data/Applications_Converted.csv", index_col=0)

## LOAD FEEDBACK
prolific_export_filePath = folder+"data\\collected_data\\prolific_export_demographics.csv"
interaction_logs_filePath = folder+"data\\collected_data\\LOGS\\"
logs_df = get_all_participants_logs(prolific_export_filePath, interaction_logs_filePath)
feedback_df = get_all_participants_feedback(logs_df)
feedback_df.drop(columns = "Lower bound",inplace = True)
feedback_df.drop(columns = "Upper bound",inplace = True)

## set new proposed distribution of feature weights in Value column
## get initial distribution
feature_weights_dict = {}
for i in range(49):
    feature_weights_dict[applications_df['ft_name_'+str(i)].tolist()[0]] = applications_df['ft_weight_'+str(i)].tolist()[0]
feature_weights = []
for feat in training_attr_descriptive_in_order:
    feature_weights.append(feature_weights_dict[feat])
## calculate new distribution
all_new_values = []
pred_decision = []
for i in feedback_df.index :
    app_id = feedback_df['App ID'].loc[i]
    p_id = feedback_df['ID'].loc[i]
    value = feedback_df['Value'].loc[i]
    ## PREDICTED DECISION
    pred_decision.append(applications_df[applications_df["Application_id"] == app_id]["Predicted_decision"].tolist()[0])
    ##
    new_value = feature_weights.copy()
    if value:
        for v_dict in value:
            attr = v_dict['attribute']
            new_value[training_attr_descriptive_in_order.index(attr)]=np.float64(v_dict['value'])
    else:
        new_value = []
    ## normalize
    new_value = [v/sum(new_value) for v in new_value]
    # if sum(new_value) >1.0:
    #     print(sum(new_value), max(new_value),min(new_value))
    all_new_values.append(new_value)    
feedback_df['Value'] = all_new_values
feedback_df['PredictedDecision'] = pred_decision

In [53]:
feedback_df

,serial number,timestamp,ID,App ID,Function,Attribute,Value,PredictedDecision
3,8,2022-12-20 14:39:40.335007,5e909231933994023e8b3c4d,106823,select_reject_application,fair,[],Accepted
5,13,2022-12-20 14:39:53.711255,5e909231933994023e8b3c4d,106647,select_reject_application,unfair,[],Rejected
7,18,2022-12-20 14:40:32.725726,5e909231933994023e8b3c4d,106401,select_reject_application,unfair,[],Rejected
9,22,2022-12-20 14:41:09.721315,5e909231933994023e8b3c4d,106861,select_reject_application,unfair,[],Rejected
11,27,2022-12-20 14:41:23.548989,5e909231933994023e8b3c4d,100001,select_reject_application,fair,[],Accepted
...,...,...,...,...,...,...,...,...
3992,94,2022-12-20 16:50:41.065143,5d66f23706e589000157e7ec,100241,select_reject_application,fair,"[0.011422637590861887, 0.06022845275181722, 0....",Rejected
3994,111,2022-12-20 16:52:09.799066,5d66f23706e589000157e7ec,106802,select_reject_application,fair,"[0.010446343779677111, 0.04653371320037986, 0....",Rejected
3996,131,2022-12-20 16:53:28.748659,5d66f23706e589000157e7ec,105035,select_reject_application,fair,"[0.010377358490566034, 0.04905660377358489, 0....",Rejected
3998,150,2022-12-20 16:54:36.783552,5d66f23706e589000157e7ec,104860,select_reject_application,fair,"[0.010638297872340422, 0.05222437137330753, 0....",Rejected


In [52]:
feedback_df_no_app_dupl_per_part = pd.DataFrame(feedback_df)

feedback_df_no_app_dupl_per_part

,serial number,timestamp,ID,App ID,Function,Attribute,Value,PredictedDecision,TMP
3,8,2022-12-20 14:39:40.335007,5e909231933994023e8b3c4d,106823,select_reject_application,fair,[],Accepted,None
5,13,2022-12-20 14:39:53.711255,5e909231933994023e8b3c4d,106647,select_reject_application,unfair,[],Rejected,None
7,18,2022-12-20 14:40:32.725726,5e909231933994023e8b3c4d,106401,select_reject_application,unfair,[],Rejected,None
9,22,2022-12-20 14:41:09.721315,5e909231933994023e8b3c4d,106861,select_reject_application,unfair,[],Rejected,None
11,27,2022-12-20 14:41:23.548989,5e909231933994023e8b3c4d,100001,select_reject_application,fair,[],Accepted,None
...,...,...,...,...,...,...,...,...,...
3992,94,2022-12-20 16:50:41.065143,5d66f23706e589000157e7ec,100241,select_reject_application,fair,"[0.011422637590861887, 0.06022845275181722, 0....",Rejected,None
3994,111,2022-12-20 16:52:09.799066,5d66f23706e589000157e7ec,106802,select_reject_application,fair,"[0.010446343779677111, 0.04653371320037986, 0....",Rejected,None
3996,131,2022-12-20 16:53:28.748659,5d66f23706e589000157e7ec,105035,select_reject_application,fair,"[0.010377358490566034, 0.04905660377358489, 0....",Rejected,None
3998,150,2022-12-20 16:54:36.783552,5d66f23706e589000157e7ec,104860,select_reject_application,fair,"[0.010638297872340422, 0.05222437137330753, 0....",Rejected,None


In [60]:
feedback_df_no_app_dupl_per_part[feedback_df_no_app_dupl_per_part['ID']=='5d66f23706e589000157e7ec']

,serial number,timestamp,ID,App ID,Function,Attribute,Value,PredictedDecision,TMP
3984,22,2022-12-20 16:43:23.782839,5d66f23706e589000157e7ec,106647,select_reject_application,unfair,"[0.011201629327902236, 0.0590631364562118, 0.0...",Rejected,None
3986,47,2022-12-20 16:45:39.529456,5d66f23706e589000157e7ec,106401,select_reject_application,fair,"[0.011689691817215724, 0.061636556854410184, 0...",Rejected,None
3988,70,2022-12-20 16:48:31.616248,5d66f23706e589000157e7ec,106488,select_reject_application,unfair,"[0.010638297872340422, 0.051257253384912944, 0...",Rejected,None
-34,None,None,5d66f23706e589000157e7ec,106401,select_reject_application,fair,"[0.011351909184726517, 0.05985552115583074, 0....",Rejected,None
3990,82,2022-12-20 16:49:48.235499,5d66f23706e589000157e7ec,101367,select_reject_application,fair,"[0.010826771653543303, 0.04232283464566928, 0....",Rejected,None
3992,94,2022-12-20 16:50:41.065143,5d66f23706e589000157e7ec,100241,select_reject_application,fair,"[0.011422637590861887, 0.06022845275181722, 0....",Rejected,None
3994,111,2022-12-20 16:52:09.799066,5d66f23706e589000157e7ec,106802,select_reject_application,fair,"[0.010446343779677111, 0.04653371320037986, 0....",Rejected,None
3996,131,2022-12-20 16:53:28.748659,5d66f23706e589000157e7ec,105035,select_reject_application,fair,"[0.010377358490566034, 0.04905660377358489, 0....",Rejected,None
3998,150,2022-12-20 16:54:36.783552,5d66f23706e589000157e7ec,104860,select_reject_application,fair,"[0.010638297872340422, 0.05222437137330753, 0....",Rejected,None
4000,165,2022-12-20 16:55:55.457697,5d66f23706e589000157e7ec,103570,select_reject_application,unfair,"[0.010784313725490192, 0.05392156862745097, 0....",Rejected,None


In [61]:
len(feedback_df_no_app_dupl_per_part[feedback_df_no_app_dupl_per_part['ID']=='5d66f23706e589000157e7ec']['App ID'].unique())

9

## Data loading

In [ ]:
## load training, validation and test sets
train_df_train = pd.read_csv(folder+'data/processed_data/train_df_train.csv', delimiter=',')
train_df_test = pd.read_csv(folder+'data/processed_data/train_df_test.csv', delimiter=',')
test_df = pd.read_csv(folder+'data/processed_data/test_df_final.csv', delimiter=',')

X_train_original = train_df_train.loc[:, ~train_df_train.columns.isin(['TARGET', 'SK_ID_CURR'])]
y_train_original = train_df_train.loc[:, train_df_train.columns == 'TARGET']
X_test_original = train_df_test.loc[:, ~train_df_test.columns.isin(['TARGET', 'SK_ID_CURR'])] ## dataset used for accuracy
y_test_original = train_df_test.loc[:, train_df_test.columns == 'TARGET']

## attributes used in training in correct order
training_attr_in_order = X_train_original.columns.tolist()
training_attr_descriptive_in_order = [attributes_names_mapping[attr[:-3]] if attr.endswith('_LE') else attributes_names_mapping[attr] for attr in training_attr_in_order]

## load original test set to be used for fairness metrics
train_df = pd.read_csv(folder+'data/processed_data/train_df.csv', delimiter=',')
test_df_app_ids = train_df_test['SK_ID_CURR'].tolist()
train_df_test_or = train_df[train_df['SK_ID_CURR'].isin(test_df_app_ids)]
train_df_test_or = train_df_test_or.reset_index().set_index('SK_ID_CURR').loc[test_df_app_ids].reset_index().set_index('index').rename_axis(None)
manipulate_categ_values(train_df_test_or)
train_df_test_bin = binning(train_df_test_or, train_df_test)

## load applications' data shown to participants through the UI
applications_df = pd.read_csv(folder+"data/processed_data/Applications_Converted.csv", index_col=0)

## LOAD FEEDBACK
prolific_export_filePath = folder+"data\\collected_data\\prolific_export_demographics.csv"
interaction_logs_filePath = folder+"data\\collected_data\\LOGS\\"
logs_df = get_all_participants_logs(prolific_export_filePath, interaction_logs_filePath)
feedback_df = get_all_participants_feedback(logs_df)
feedback_df.drop(columns = "Lower bound",inplace = True)
feedback_df.drop(columns = "Upper bound",inplace = True)

## set new proposed distribution of feature weights in Value column
## get initial distribution
feature_weights_dict = {}
for i in range(49):
    feature_weights_dict[applications_df['ft_name_'+str(i)].tolist()[0]] = applications_df['ft_weight_'+str(i)].tolist()[0]
feature_weights = []
for feat in training_attr_descriptive_in_order:
    feature_weights.append(feature_weights_dict[feat])
## calculate new distribution
all_new_values = []
pred_decision = []
for i in feedback_df.index :
    app_id = feedback_df['App ID'].loc[i]
    p_id = feedback_df['ID'].loc[i]
    value = feedback_df['Value'].loc[i]
    ## PREDICTED DECISION
    pred_decision.append(applications_df[applications_df["Application_id"] == app_id]["Predicted_decision"].tolist()[0])
    ##
    new_value = feature_weights.copy()
    if value:
        for v_dict in value:
            attr = v_dict['attribute']
            new_value[training_attr_descriptive_in_order.index(attr)]=np.float64(v_dict['value'])
    else:
        new_value = []
    ## normalize
    new_value = [v/sum(new_value) for v in new_value]
    # if sum(new_value) >1.0:
    #     print(sum(new_value), max(new_value),min(new_value))
    all_new_values.append(new_value)    
feedback_df['Value'] = all_new_values
feedback_df['PredictedDecision'] = pred_decision

## TRAINING

### Global Model - Labels ('fair'+'unfair')
The XGboost classifier is retrained. The feedback instances of all participants are added into the training set all at once.

**All the 'fair' and 'unfair'** labelled instances by participants are used. 

We set fs = 0.5.

In [ ]:
oneoff_training_evaluation(X_train_original, y_train_original,
                            X_test_original, y_test_original,
                           train_df_test_bin, test_df,
                            sensitive_attrs, fs, feedback_df,
                           False, False,
                           folder+"data/results/global/Labels/", 'global-Labels')

### Global Model - Labels ('unfair')
The XGboost classifier is retrained. The feedback instances of all participants are added into the training set all at once.

**Only the 'unfair'** labelled instances by participants are used. 

We set fs = 0.5.

In [ ]:
oneoff_training_evaluation(X_train_original, y_train_original,
                            X_test_original, y_test_original,
                           train_df_test_bin, test_df,
                            sensitive_attrs, fs, feedback_df,
                           True, False,
                           folder+"data/results/global/Labels_Unfair/", 'global-Labels_Unfair')

### Global Model - Labels+Weights ('fair'+'unfair')
The XGboost classifier is retrained. The feedback instances of all participants are added into the training set all at once.

**All the 'fair' and 'unfair'** labelled instances by participants **with proposed weight changes** are used. 

We set fs = 0.5.

In [ ]:
oneoff_training_evaluation(X_train_original, y_train_original,
                            X_test_original, y_test_original,
                           train_df_test_bin, test_df,
                            sensitive_attrs, fs, feedback_df,
                           False, True, 
                           folder+"data/results/global/Labels+Weights/", 'global-Labels+Weights')

### Personalized Models - Labels ('fair'+'unfair')
58 XGboost classifiers will be trained, one for each participant independently. The feedback instances of each participant will be added to the training set incrementally, one at a time, in increasing timestamps (the previous feedback instances of a participant remain in the training set as we add a new one). The XGboost classifier is retrained from scratch in every incremental step (iteration). 

**All the 'fair' and 'unfair'** labelled instances by participants are used. 

We set fs = 0.5.

In [ ]:
iml_training_evaluation(X_train_original, y_train_original,
                        X_test_original, y_test_original,
                        train_df_test_bin, test_df,
                        sensitive_attrs, fs, feedback_df,
                        False, False, 
                        folder+"data/results/personalized/Labels/", 'personalized-Labels')

### Personalized Models - Labels ('unfair')
58 XGboost classifiers will be trained, one for each participant independently. The feedback instances of each participant will be added to the training set incrementally, one at a time, in increasing timestamps (the previous feedback instances of a participant remain in the training set as we add a new one). The XGboost classifier is retrained from scratch in every incremental step (iteration). 

**Only the 'unfair'** labelled instances by participants are used.

We set fs = 0.5.

In [ ]:
iml_training_evaluation(X_train_original, y_train_original,
                        X_test_original, y_test_original,
                        train_df_test_bin, test_df,
                        sensitive_attrs, fs, feedback_df,
                        True, False, 
                        folder+"data/results/personalized/Labels_Unfair/", 'personalized-Labels_Unfair')

### Personalized Models - Labels+Weights ('fair'+'unfair')
The feedback instances of each participant will be added to the training set incrementally, one at a time, in increasing timestamps (the previous feedback instances of a participant remain in the training set as we add a new one). The XGboost classifier is retrained from scratch in every incremental step (iteration).

**All the 'fair' and 'unfair'** labelled instances with proposed weight changes are used. Participants weights of each instance are used to set feature weights of the training.

We set fs = 0.5.

In [ ]:
iml_training_evaluation(X_train_original, y_train_original,
                        X_test_original, y_test_original,
                        train_df_test_bin, test_df,
                        sensitive_attrs, fs, feedback_df,
                        False, True, 
                        folder+"data/results/personalized/Labels+Weights/", 'personalized-Labels+Weights')